## cleaning apartments posts for sale

In [1]:
import pandas as pd
import numpy as np
import re

from IPython.display import Image, display #for tree plot 
import pydotplus 
from scipy import misc

import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

### import the data

In [2]:
df=pd.read_csv('PATH TO FILE HERE',sep=',',low_memory=False)

In [14]:
df.shape

(28777, 24)

### clean Nans

In [3]:
df_cln_sale=df

df_cln_sale['balcony'].fillna(0,inplace=True)
df_cln_sale['building_floors'].fillna(0,inplace=True)
df_cln_sale['sub_area'].fillna('לא צוין',inplace=True)
df_cln_sale=df_cln_sale.drop(df_cln_sale.loc[:,list((100*(df_cln_sale.isnull().sum()/len(df_cln_sale.index))>40))].columns, 1)

### drop undesire columns

In [4]:
df_cln_sale=df_cln_sale.drop(df.columns[0], axis=1)
df_cln_sale=df_cln_sale.drop(df.columns[9], axis=1)

### change data types

In [5]:
df_cln_sale['state_of_asset'].replace(['חדש מקבלן'],['חדש'],inplace=True)

df_cln_sale=df_cln_sale.drop(df_cln_sale.index[df_cln_sale['rooms']=='-'])
df_cln_sale['rooms']=df_cln_sale['rooms'].astype(np.float)

df_cln_sale['floor'].replace(['קרקע'],0,inplace=True)
df_cln_sale['floor']=df_cln_sale['floor'].astype(np.int64)

df_cln_sale['size'].replace(['לא צוין'],None,inplace=True)
df_cln_sale['size'].replace(['לא צוין'],df_cln_sale['size'].mean(),inplace=True)
df_cln_sale['size']=df_cln_sale['size'].astype(np.int64)

df_cln_sale['price'].replace(['לאצויןמחיר'],None,inplace=True)
df_cln_sale['price'].replace(['לאצויןמחיר'],df_cln_sale['price'].mean(),inplace=True)
df_cln_sale['price']=df_cln_sale['price'].astype(np.int64)

df_cln_sale['balcony']=df_cln_sale['balcony'].astype(np.int64)

df_cln_sale['parkings'].replace(['ללא'],0,inplace=True)
df_cln_sale['parkings']=df_cln_sale['parkings'].astype(np.int64)

df_cln_sale['date_update'] = pd.to_datetime(df_cln_sale['date_update'],dayfirst=True)

In [6]:
#df_cln_sale['date_update'].value_counts()

In [7]:
df_cln_sale.isna().sum()
df_cln_sale.dtypes

adress                     object
aprtment_type              object
sub_area                   object
city                       object
rooms                     float64
floor                       int64
size                        int64
price                       int64
balcony                     int64
building_floors           float64
state_of_asset             object
parkings                    int64
is_aircon                   int64
is_borderwin                int64
is_elevator                 int64
is_renovated                int64
is_storeroom                int64
is_furneter                 int64
date_update        datetime64[ns]
is_sold                     int64
dtype: object

### outliers

In [8]:
df_cln_sale=df_cln_sale[df_cln_sale['floor']<100]

df_cln_sale.loc[df_cln_sale['size']>=5000]=np.nan
df_cln_sale.loc[df_cln_sale['size']<=25]=np.nan
df_cln_sale['size'].fillna(df_cln_sale['size'].median(),inplace=True)

df_cln_sale=df_cln_sale[df_cln_sale['price']>=100000]
df_cln_sale=df_cln_sale[df_cln_sale['price']<=20000000]

In [9]:
#check what the range value of the outliers(with median and IQR)
#score = (df_cln_sale['colname']-df_cln_sale['colname'].mean()) / df_cln_sale['colname'].std()
#outliers = abs(score) > 3
#sum(outliers)

#Q1 = np.percentile(df_cln_sale['colname'], 25)
#Q3 = np.percentile(df_cln_sale['colname'], 75)
#IQR = Q3-Q1
#price_outlier_rows = df_cln_sale[(df_cln_sale['colname'] < Q1 - 1.5*IQR) | (df_cln_sale['colname'] > Q3 + 1.5*IQR)].index
#len(price_outlier_rows)
#df_cln_sale.loc[price_outlier_rows]

### fixing categorical features

In [10]:
df_cln_sale['aprtment_type'].value_counts()

דירה          18806
פרטי/קוטג'     4497
דירת גן        2047
גג/פנטהאוז     1684
דו משפחתי      1227
יחידת דיור        8
Name: aprtment_type, dtype: int64

In [11]:
#clean less common cities (less than 20)
temp=df_cln_sale['city'].value_counts()
val_to_keep=temp.loc[temp>=20]
df_cln_sale= df_cln_sale[df_cln_sale['city'].isin(val_to_keep.index.tolist())]

In [12]:
df_cln_sale['city'].value_counts()

תל אביב יפו    2418
ירושלים        1686
חיפה           1507
באר שבע        1352
ראשון לציון     978
               ... 
שלומי            24
קרני שומרון      24
כפר תבור         23
כפר ורדים        23
אבני חפץ         22
Name: city, Length: 106, dtype: int64

In [13]:
df_cln_sale.shape

(27035, 20)

In [13]:
df_cln_sale.to_csv('forsale_clean.csv')